In [1]:
import os
import streamlit as st
os.environ["ANTHROPIC_API_KEY"] = st.secrets["ANTHROPIC_API_KEY"]
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [3]:
# Docstring is required, make it good!

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [multiply, exponentiate, add]

In [4]:
llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what's 3 plus 5 raised to the power of 2. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2}`
responded: [{'text': 'To calculate 3 + 5^2, we can use the "exponentiate" tool first:', 'type': 'text', 'index': 0}, {'id': 'toolu_01WyjnKsUfw9kyf4Wwq3WT6V', 'input': {}, 'name': 'exponentiate', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 5, "y": 2}'}]

25.0
Invoking: `add` with `{'x': 3, 'y': 25}`
responded: [{'text': '\n\n5 raised to the power of 2 is 25.\n\nThen we can use the "add" tool:', 'type': 'text', 'index': 0}, {'id': 'toolu_01PqBYDVeNk5pQbDpH7vdkPi', 'input': {}, 'name': 'add', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 3, "y": 25}'}]

28.0
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`
responded: [{'text': '\n\nSo 3 + 5^2 = 28\n\nFor the second calculation 17.24 - 918.1241, we can use:', 'type': 'text', 'index': 0}, {'id': 'toolu_01GcZAU6ece9y6V2QTzU3QCF', 'input': {}, 'name': 'add', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 17.24, "y": -918.12

{'input': "what's 3 plus 5 raised to the power of 2. also what's 17.24 - 918.1241",
 'output': [{'text': '\n\nTherefore, 17.24 - 918.1241 = -900.8841',
   'type': 'text',
   'index': 0}]}

In [7]:
llm = ChatOpenAI(model = "gpt-4", temperature=0)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what's 3 plus the quantity 5 raised to the power of 2. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2}`
responded: To solve the first part of your question, we need to first calculate 5 raised to the power of 2 and then add 3 to the result. For the second part, we simply subtract 918.1241 from 17.24. Let me do the calculations.

25.0
Invoking: `add` with `{'x': 3, 'y': 25}`


28.0
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`
responded: The result of 3 plus the quantity 5 raised to the power of 2 is 28.

Now, let's calculate 17.24 - 918.1241.

-900.8841The result of 17.24 - 918.1241 is -900.8841.

> Finished chain.


{'input': "what's 3 plus the quantity 5 raised to the power of 2. also what's 17.24 - 918.1241",
 'output': 'The result of 17.24 - 918.1241 is -900.8841.'}

In [13]:
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what's 3 plus 5 raised to the power of 2", })



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 3, 'y': 5}`


8.0
Invoking: `exponentiate` with `{'x': 5, 'y': 2}`


25.0The result of \(3 + 5\) is \(8\), and \(5\) raised to the power of \(2\) is \(25\).

> Finished chain.


{'input': "what's 3 plus 5 raised to the power of 2",
 'output': 'The result of \\(3 + 5\\) is \\(8\\), and \\(5\\) raised to the power of \\(2\\) is \\(25\\).'}

In [14]:
llm = ChatOllama(
    model="llama3-groq-tool-use:70b",
   # model="llama3.1",
    temperature=0,
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({
    "system": "You are a helpful agent.  Use the exponentiate tool to compute exponents (powers) and the add tool to compute addition and subtraction. Always try to determine and invoke a tool to perform your tasks.  If you do not have enough information, ask for more details.",
    "input": "what's 3 plus the value of 5 raised to the 2.743. Then substract 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`


82.65606421491815
Invoking: `add` with `{'x': 3, 'y': 82.65606421491815}`


85.65606421491815
Invoking: `add` with `{'x': -918.1241, 'y': 85.65606421491815}`


-832.4680357850818The final result is approximately -832.47.
 Note that the provided function is in Python.

> Finished chain.


{'system': 'You are a helpful agent.  Use the exponentiate tool to compute exponents (powers) and the add tool to compute addition and subtraction. Always try to determine and invoke a tool to perform your tasks.  If you do not have enough information, ask for more details.',
 'input': "what's 3 plus the value of 5 raised to the 2.743. Then substract 918.1241",
 'output': 'The final result is approximately -832.47.\n Note that the provided function is in Python.'}